In [1]:
import cv2
import numpy as np
from tkinter import *
from tkinter import filedialog, messagebox, ttk
from tkinter.ttk import Style, Progressbar, Treeview
from PIL import Image, ImageTk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from openpyxl import Workbook, load_workbook
from datetime import datetime
import os

uploaded_image_path = None
last_result = ""
last_percent = 0
webcam_running = False

# Warna tema dan font
BG_COLOR = "#2F4858"
PRIMARY_COLOR = "#336699"
SECONDARY_COLOR = "#86BBD8"
ACCENT_COLOR = "#FF6347"
TEXT_COLOR = "#2F4F4F"
BUTTON_HOVER = "#5F9EA0"
FRAME_COLOR = "#E6E6FA"
FONT_TITLE = ("Poppins", 14, "bold")
FONT_LABEL = ("Poppins", 11)
FONT_RESULT = ("Poppins", 12, "bold")


def show_histogram(image_path):
    image = cv2.imread(image_path)
    color = ('b', 'g', 'r')
    fig = Figure(figsize=(4, 3), dpi=100)
    plot = fig.add_subplot(111)
    for i, col in enumerate(color):
        histr = cv2.calcHist([image], [i], None, [256], [0, 256])
        plot.plot(histr, color=col)
    plot.set_title('Histogram Warna', color=TEXT_COLOR)
    plot.set_facecolor("#FFFFFF")
    fig.patch.set_facecolor("#FFFFFF")
    for spine in plot.spines.values():
        spine.set_edgecolor(TEXT_COLOR)
    canvas = FigureCanvasTkAgg(fig, master=histogram_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()


def detect_ripeness(image, fruit_type):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    color_ranges = {
        "Tomat": ([0, 100, 100], [10, 255, 255]),
        "Mangga": ([30, 100, 100], [60, 255, 255]),
        "Strawberry": ([0, 50, 50], [10, 255, 255]),
        "Kiwi": ([30, 50, 50], [90, 255, 255]),
        "Alpukat": ([40, 50, 50], [80, 255, 255])
    }

    if fruit_type not in color_ranges:
        return "Jenis buah tidak dikenali", 0, image

    lower, upper = np.array(color_ranges[fruit_type][0]), np.array(color_ranges[fruit_type][1])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        total_area = image.shape[0] * image.shape[1]
        detected_area = cv2.countNonZero(mask)
        percent = round((detected_area / total_area) * 100, 2)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        hasil = f"{fruit_type} Matang ({percent}%)" if percent > 15 else f"{fruit_type} Belum Matang ({percent}%)"
        return hasil, percent, image

    return f"{fruit_type} Tidak Terdeteksi", 0, image


def simpan_hasil_otomatis(fruit, percent, metode):
    filename = "hasil_deteksi.xlsx"
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    if not os.path.exists(filename):
        wb = Workbook()
        ws = wb.active
        ws.append(["Waktu", "Jenis Buah", "Persentase Kematangan", "Metode"])
    else:
        wb = load_workbook(filename)
        ws = wb.active
    ws.append([waktu, fruit, f"{percent}%", metode])
    wb.save(filename)


def upload_image():
    global uploaded_image_path
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    uploaded_image_path = file_path
    img = Image.open(file_path).resize((300, 300))
    img_tk = ImageTk.PhotoImage(img)
    img_label.config(image=img_tk)
    img_label.image = img_tk
    result_label.config(text="Klik 'Scan Kematangan' untuk deteksi", fg=TEXT_COLOR)
    for widget in histogram_frame.winfo_children():
        widget.destroy()
    show_histogram(file_path)


def scan_image():
    global last_result, last_percent
    for widget in histogram_frame.winfo_children():
        widget.destroy()
    if uploaded_image_path:
        selected_fruit = fruit_var.get()
        image = cv2.imread(uploaded_image_path)
        result, percent, processed_image = detect_ripeness(image, selected_fruit)
        last_result, last_percent = result, percent
        warna = SECONDARY_COLOR if "Matang" in result else ACCENT_COLOR
        result_label.config(text=f"Hasil: {result}", fg=warna, font=FONT_RESULT)
        progress_bar['value'] = percent
        processed_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(processed_image).resize((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        img_label.config(image=img_tk)
        img_label.image = img_tk
        show_histogram(uploaded_image_path)
        simpan_hasil_otomatis(selected_fruit, percent, "Upload Gambar")
        load_data_to_table()
    else:
        messagebox.showinfo("Info", "Silakan upload gambar terlebih dahulu!")


def save_processed_image():
    if last_result and uploaded_image_path:
        name = f"hasil_{os.path.basename(uploaded_image_path)}"
        save_path = filedialog.asksaveasfilename(defaultextension=".png", initialfile=name)
        if save_path:
            image = cv2.imread(uploaded_image_path)
            _, _, processed = detect_ripeness(image, fruit_var.get())
            cv2.imwrite(save_path, processed)
            messagebox.showinfo("Berhasil", f"Gambar disimpan di {save_path}")


def detect_with_webcam():
    global webcam_running
    if webcam_running:
        webcam_running = False
        webcam_btn.config(text="Webcam Deteksi")
        return
    webcam_running = True
    webcam_btn.config(text="Stop Webcam")
    selected_fruit = fruit_var.get()
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Webcam tidak ditemukan!")
        webcam_running = False
        webcam_btn.config(text="Webcam Deteksi")
        return

    def update_webcam():
        if webcam_running:
            ret, frame = cap.read()
            if ret:
                frame = cv2.flip(frame, 1)
                result, percent, processed_frame = detect_ripeness(frame.copy(), selected_fruit)
                cv2.putText(processed_frame, result, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(processed_frame, f"{percent}%", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                warna = SECONDARY_COLOR if "Matang" in result else ACCENT_COLOR
                result_label.config(text=f"Hasil: {result}", fg=warna, font=FONT_RESULT)
                progress_bar['value'] = percent
                processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(processed_frame).resize((300, 300))
                img_tk = ImageTk.PhotoImage(img)
                img_label.config(image=img_tk)
                img_label.image = img_tk
                img_label.after(10, update_webcam)
            else:
                stop_webcam()
        else:
            stop_webcam()

    def stop_webcam():
        nonlocal cap
        if cap:
            cap.release()
            cv2.destroyAllWindows()
        webcam_running = False
        webcam_btn.config(text="Webcam Deteksi")

    update_webcam()


def load_data_to_table():
    for item in riwayat_table.get_children():
        riwayat_table.delete(item)
    if os.path.exists("hasil_deteksi.xlsx"):
        wb = load_workbook("hasil_deteksi.xlsx")
        ws = wb.active
        for row in ws.iter_rows(min_row=2, values_only=True):
            riwayat_table.insert('', 'end', values=row)


# GUI
root = Tk()
root.title("Deteksi Kematangan Buah")
root.geometry("1000x800")
root.configure(bg=BG_COLOR)

# Header dengan logo dan judul
header_frame = Frame(root, bg=BG_COLOR)
header_frame.pack(fill=X, pady=10)

try:
    logo_img = Image.open("images/logo to.png").resize((50, 50))  # Ganti "logo.png" dengan nama file logo kamu
    logo_img_tk = ImageTk.PhotoImage(logo_img)
    logo_label = Label(header_frame, image=logo_img_tk, bg=BG_COLOR)
    logo_label.image = logo_img_tk  # Simpan referensinya agar tidak dihapus oleh garbage collector
    logo_label.pack(side=LEFT, padx=10)
except Exception as e:
    print("Logo tidak ditemukan atau gagal dimuat:", e)

judul_label = Label(header_frame, text="Pendeteksi Kematangan Buah | Kelompok 4", font=("Poppins", 18, "bold"), bg=BG_COLOR, fg="white")
judul_label.pack(side=LEFT, padx=10)

style = Style()
style.theme_use('clam')
style.configure('green.Horizontal.TProgressbar', troughcolor=BG_COLOR, background=SECONDARY_COLOR)
style.configure('red.Horizontal.TProgressbar', troughcolor=BG_COLOR, background=ACCENT_COLOR)

notebook = ttk.Notebook(root)
notebook.pack(fill=BOTH, expand=True, padx=20, pady=10)

tab_deteksi = Frame(notebook, bg=BG_COLOR)
tab_riwayat = Frame(notebook, bg=BG_COLOR)
notebook.add(tab_deteksi, text="Deteksi Buah")
notebook.add(tab_riwayat, text="Riwayat Deteksi")

fruit_var = StringVar(value="Alpukat")

content_frame = Frame(tab_deteksi, bg=BG_COLOR)
content_frame.pack(fill=BOTH, expand=True)

left_panel = Frame(content_frame, bg=BG_COLOR)
left_panel.pack(side=LEFT, fill=BOTH, expand=True, padx=10, pady=10)

img_label = Label(left_panel, bg="white", relief="ridge", height=20)
img_label.pack(pady=10, padx=5, fill=BOTH, expand=True)

histogram_frame = Frame(left_panel, bg=BG_COLOR)
histogram_frame.pack(pady=10, padx=5, fill=BOTH, expand=True)

right_panel = Frame(content_frame, bg=BG_COLOR)
right_panel.pack(side=RIGHT, fill=BOTH, expand=True, padx=10, pady=10)

result_label = Label(right_panel, text="Klik 'Scan Kematangan'", font=FONT_LABEL, bg="white", relief="ridge")
result_label.pack(pady=10, padx=5, fill=X)

progress_bar = Progressbar(right_panel, orient=HORIZONTAL, length=300, mode='determinate', style='red.Horizontal.TProgressbar')
progress_bar.pack(pady=10, padx=5, fill=X)

option_frame = Frame(tab_deteksi, bg=BG_COLOR)
option_frame.pack(pady=10)
Label(option_frame, text="Pilih Buah:", font=FONT_LABEL, bg=BG_COLOR).pack(side=LEFT, padx=5)
OptionMenu(option_frame, fruit_var, "Tomat", "Mangga", "Strawberry", "Kiwi", "Alpukat").pack(side=LEFT, padx=5)

button_frame = Frame(tab_deteksi, bg=BG_COLOR)
button_frame.pack(pady=10)
Button(button_frame, text="Upload Gambar", command=upload_image, bg=PRIMARY_COLOR, fg="white", font=FONT_LABEL).pack(side=LEFT, padx=5)
Button(button_frame, text="Scan Kematangan", command=scan_image, bg=SECONDARY_COLOR, fg="white", font=FONT_LABEL).pack(side=LEFT, padx=5)
webcam_btn = Button(button_frame, text="Webcam Deteksi", command=detect_with_webcam, bg="#9370DB", fg="white", font=FONT_LABEL)
webcam_btn.pack(side=LEFT, padx=5)
Button(button_frame, text="Simpan Gambar", command=save_processed_image, bg=PRIMARY_COLOR, fg="white", font=FONT_LABEL).pack(side=LEFT, padx=5)

table_frame = Frame(tab_riwayat, bg=BG_COLOR)
table_frame.pack(fill=BOTH, expand=True, padx=10, pady=10)
riwayat_table = Treeview(table_frame, columns=("Waktu", "Buah", "Persentase", "Metode"), show='headings')
for col in ("Waktu", "Buah", "Persentase", "Metode"):
    riwayat_table.heading(col, text=col)
    riwayat_table.column(col, width=120, anchor=CENTER)
riwayat_table.pack(fill=BOTH, expand=True)
Button(tab_riwayat, text="Refresh Tabel", command=load_data_to_table, bg="#FFA500", fg="white", font=FONT_LABEL).pack(pady=5)

load_data_to_table()
root.mainloop()
